# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
weather_data= os.path.join("../output_data/weather_data.csv")

vacation_data = pd.read_csv(weather_data)
vacation_data

,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Windspeed,Country,Date
0,albany,42.6001,-73.9662,-11.25,62,26,3.57,US,1644894018
1,clyde river,70.4692,-68.5914,-29.44,94,100,1.61,CA,1644894087
2,urucara,-2.5364,-57.7600,22.74,98,100,0.94,BR,1644894087
3,bluff,-46.6000,168.3333,21.56,61,80,3.05,NZ,1644894087
4,saskylakh,71.9167,114.0833,-29.32,87,75,0.54,RU,1644894087
...,...,...,...,...,...,...,...,...,...
561,melbu,68.5044,14.8297,-3.20,67,14,1.38,NO,1644894291
562,quirinopolis,-18.4483,-50.4517,21.66,90,6,1.63,BR,1644894291
563,yatou,37.1500,122.3833,-1.87,43,89,6.86,CN,1644894292
564,jhanjharpur,26.2667,86.2833,17.53,54,0,2.28,IN,1644894292


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
gmaps.configure(api_key=g_key)

In [32]:
locations = vacation_data[["Lat", "Lng"]]
weights = vacation_data ["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False)
heat_layer.max_intensity = 75
heat_layer.point_radius = 2


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
hotel_df = vacation_data[(vacation_data['Max_temp'] >= 25) & (vacation_data['Max_temp'] <= 75)]
hotel_df= hotel_df[(hotel_df['Windspeed'] <= 10) & (hotel_df['Cloudiness'] <= 10) & (hotel_df['Humidity'] <= 75)]
hotel_df

,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Windspeed,Country,Date
23,busselton,-33.6500,115.3333,32.25,45,0,2.67,AU,1644894094
39,atuona,-9.8000,-139.0333,25.67,73,2,6.45,PF,1644894100
48,carnarvon,-24.8667,113.6333,31.87,55,4,4.71,AU,1644894110
92,geraldton,-28.7667,114.6000,36.23,34,2,5.66,AU,1644894125
124,san rafael,-34.6177,-68.3301,25.61,27,0,2.24,AR,1644894135
154,bengkulu,-3.8004,102.2655,28.77,62,8,1.05,ID,1644894146
159,touros,-5.1989,-35.4608,26.64,75,1,5.52,BR,1644894147
180,kavaratti,10.5669,72.6420,26.92,71,5,2.16,IN,1644894154
237,conde,-7.2597,-34.9075,25.18,74,3,3.64,BR,1644894136
270,lujan,-34.5703,-59.1050,25.55,70,0,2.71,AR,1644894186


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df["Hotel_name"]=""
hotel_df

,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel_name
23,busselton,-33.6500,115.3333,32.25,45,0,2.67,AU,1644894094,
39,atuona,-9.8000,-139.0333,25.67,73,2,6.45,PF,1644894100,
48,carnarvon,-24.8667,113.6333,31.87,55,4,4.71,AU,1644894110,
92,geraldton,-28.7667,114.6000,36.23,34,2,5.66,AU,1644894125,
124,san rafael,-34.6177,-68.3301,25.61,27,0,2.24,AR,1644894135,
154,bengkulu,-3.8004,102.2655,28.77,62,8,1.05,ID,1644894146,
159,touros,-5.1989,-35.4608,26.64,75,1,5.52,BR,1644894147,
180,kavaratti,10.5669,72.6420,26.92,71,5,2.16,IN,1644894154,
237,conde,-7.2597,-34.9075,25.18,74,3,3.64,BR,1644894136,
270,lujan,-34.5703,-59.1050,25.55,70,0,2.71,AR,1644894186,


In [35]:
hotel_df.count()

City          20
Lat           20
Lng           20
Max_temp      20
Humidity      20
Cloudiness    20
Windspeed     20
Country       20
Date          20
Hotel_name    20
dtype: int64

In [36]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        response = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel_name"] = response["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel_name"] = "NaN"
        
hotel_df        

,City,Lat,Lng,Max_temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel_name
23,busselton,-33.6500,115.3333,32.25,45,0,2.67,AU,1644894094,Esplanade Hotel
39,atuona,-9.8000,-139.0333,25.67,73,2,6.45,PF,1644894100,Hôtel Hiva Oa Hanakee Lodge
48,carnarvon,-24.8667,113.6333,31.87,55,4,4.71,AU,1644894110,Hospitality Carnarvon
92,geraldton,-28.7667,114.6000,36.23,34,2,5.66,AU,1644894125,Ocean Centre Hotel
124,san rafael,-34.6177,-68.3301,25.61,27,0,2.24,AR,1644894135,SAN MARTIN HOTEL Y SPA
154,bengkulu,-3.8004,102.2655,28.77,62,8,1.05,ID,1644894146,Hotel Santika Bengkulu
159,touros,-5.1989,-35.4608,26.64,75,1,5.52,BR,1644894147,Hotel Vila Gale Touros
180,kavaratti,10.5669,72.6420,26.92,71,5,2.16,IN,1644894154,Hotels in Lakshadweep Islands
237,conde,-7.2597,-34.9075,25.18,74,3,3.64,BR,1644894136,NaN
270,lujan,-34.5703,-59.1050,25.55,70,0,2.71,AR,1644894186,Hotel Hoxon Lujan


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))